In [2]:
#예제1

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

park = Customer('Park',100)
park

Customer(name='Park',fidelity=100)

class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
      return self.price * self.quantity

In [3]:
class Order:  # Context
    """고객(nametuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion  # 할인 객체

    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)  # self = Order 객체
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


In [5]:
class Promotion(ABC): #strategy : abstract base class
  """할인 혜택 클래스들의 형태를 선언"""

@abstractmethod
def discount(self,order):
  """할인액을 구체적인 숫자로 반환"""
  pass

In [7]:

class Promotion:
    """할인 정책의 기본 클래스"""
    def discount(self, order):
        """할인 금액을 계산해서 반환"""
        return 0


class FidelityPromo(Promotion):
    """총점 1000점 이상 고객에게 전체 5% 할인"""
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0



class BulkItemPromo(Promotion):
    """20개 이상 동일 상품 구입 시 10% 할인"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount


class LargeOrderPromo(Promotion):
    """10종류 이상 상품 구입 시 전체 7% 할인"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0


In [8]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana',4,.5),
        LineItem('apple',10,1.5),
        LineItem('watermellon',5,5.0)]

In [10]:
Order(joe,cart,FidelityPromo())

<Order total: 42.00 due: 42.00>

In [11]:
Order(ann,cart,FidelityPromo())

<Order total: 42.00 due: 39.90>

In [12]:
banana_cart = [LineItem('banana',30,.5),
               LineItem('apple',10,1.5)]

In [14]:
Order(joe,banana_cart,BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [15]:
long_order = [LineItem(str(item_code),1,1.0)
for item_code in range(10)]

In [17]:
Order(joe,long_order,LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [18]:
Order(joe,cart,LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [19]:
class Order:  # Context
    """고객(nametuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion  # 할인 함수(또는 None)

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)  # promotion은 함수 객체
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


In [20]:
def fidelity_promo(order):
    """총성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount


def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


In [22]:
Order(joe,cart,fidelity_promo)

<Order total: 42.00 due: 42.00>

In [23]:
Order(ann,cart,fidelity_promo)

<Order total: 42.00 due: 39.90>

In [24]:
Order(joe,banana_cart,bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [25]:
Order(ann,long_order,large_order_promo)

<Order total: 10.00 due: 9.30>

In [26]:
promos = [fidelity_promo,bulk_item_promo,large_order_promo]

def best_promo(order):
  """최대로 할인받을 금액을 반환"""
  return max(promo(order) for promo in promos)


In [27]:
Order(joe,long_order,best_promo)

<Order total: 10.00 due: 9.30>

In [28]:
Order(joe,banana_cart,best_promo)

<Order total: 30.00 due: 28.50>

In [29]:
Order(ann,cart,best_promo)

<Order total: 42.00 due: 39.90>

In [30]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', 'park', 'LineItem', '_i2', '_i3', 'Order', '_i4', '_i5', 'Promotion', 'discount', '_i6', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i7', '_i8', 'joe', 'ann', 'cart', '_i9', '_i10', '_10', '_i11', '_11', '_i12', 'banana_cart', '_i13', '_13', '_i14', '_14', '_i15', 'long_order', '_i16', '_i17', '_17', '_i18', '_18', '_i19', '_i20', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i21', '_i22', '_22', '_i23', '_23', '_i24', '_24', '_i25', '_25', '_i26', 'promos', 'best_promo', '_i27', '_27', '_i28', '_28', '_i29', '_29', '_i30'])

In [31]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [32]:
promos = [globals()[name]for name in globals()
if name.endswith('promo')
and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]